In [1]:
with open('../key.txt', 'r') as f:
    key_file = f.read()

key_file = key_file.split('\n')
keys = {}
for key in key_file:
    key = key.split(':')
    keys[key[0]] =  key[1]

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = keys['OPENAI_API_KEY']

# Guidance Tutorial

In [2]:
import guidance 

In [ ]:
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

## Basic Templating
- Direct template
- Lists & Objects template
- Program include programs

### Direct

In [3]:
program = guidance(
    """
    What is {{X}}?
    """
)

In [5]:
program_test = program(X = 'xxx')

Stop program 
 What is xxx ?

In [24]:
''.join([str(type(repr(program_test))),' <The Content is: ', repr(program_test), '>'])

"<class 'str'> <The Content is: \n    What is xxx?\n    >"

In [10]:
program_test['X']

'xxx'

### Lists and Objects

In [25]:
people = ['x', 'y', 'z']
weapons = [
    {'name': 'knife', 'damage': 10},
    {'name': 'sword', 'damage': 20}
]

In [36]:
template = """
List of People:
{{#each people}} - {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace}}
{{/each~}}
List of ideas:
{{#each weapons}}{{this.name}}: {{this.damage}}
{{/each}}
"""

In [37]:
program = guidance(
    template
)

executed_program = program(people = people, weapons = weapons)

Stop program 
List of People:
 - x 
 - y 
 - z 
 List of ideas:
 knife : 10 
 sword : 20

### Includes (Including guidance programs inside other programs)

In [38]:
template1 = """
List of People:
{{#each people}} - {{this}}
{{/each~}}
"""

template2 = """
{{>program1}}
List of weapons:
{{#each weapons}}{{this.name}}: {{this.damage}}
{{/each}}
"""

In [39]:
program1 = guidance(template1)
program2 = guidance(template2)

executed_program = program2(
    program1 = program1,
    people = people,
    weapons = weapons
)

Stop program 
 
List of People:
 - x 
 - y 
 - z 
 
List of weapons:
 knife : 10 
 sword : 20

## Basic command
- gen
    - guidance.llms.OpenAI.cache.clear()

- select
- Multiple Generates
- Hidden
- Await


### gen
- generating with n>1
    - the {{gen}} command the n=number argument to generate multiple completion only the first completion is used for future context, but the variable set by the command is a list of all the completions


In [46]:
template = '''
The best thing about {{X}} is {{~gen 'best' n=1 temperature=0 max_tokens=20}}
'''

In [49]:
program = guidance(template = template)

executed_program = program(X = 'doing AI Research')

Stop program 
The best thing about doing AI Research is that it is a field that is constantly evolving and growing. This means that there is always something new

In [43]:

# you can flush a cache by calling the clear method
# (this returns the number of items that were cleared)
guidance.llms.OpenAI.cache.clear()

# you can also disable caching by passing caching=False to the LLM constructor
# guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

26

### select

In [50]:
template = '''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}'''

In [54]:
# the {{#select}} command allows you to use the LLM to select from a set of options
program = guidance(template = template)
executed_program = program(example='You whore')

Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: You whore 
Answer: Yes

In [55]:
executed_program['logprobs']

{' Yes': -0.007883091000000028, ' No': -17.899223, ' Maybe': -4.8469715}

In [56]:
executed_program['answer']

' Yes'

### Multiple Generates
- every time put the content above to the LLM and Generate
    - If API, costy

In [59]:
template = '''Generate a response to the following saying:
{{saying}}.
Response:{{gen "response" temperature=0}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}'''

saying = 'I am so smart that I think Elon Musk can\'t beat me!'

In [61]:
program = guidance(
    template = template,
    steam = True,
    caching = False)
executed_program = program(saying = saying) 

Stop program Generate a response to the following saying:
 I am so smart that I think Elon Musk can't beat me! .
Response: That's quite a bold statement! I'm sure Elon Musk has a few tricks up his sleeve that could surprise you. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

In [63]:
executed_program['response']

" That's quite a bold statement! I'm sure Elon Musk has a few tricks up his sleeve that could surprise you."

### Hidden
- using hidden=True (Some not supports)
- using block command (add hidden=True in block)

In [64]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')

executed_program = program(email='I hate tacos')

Stop program 
I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

### Await

In [65]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [66]:
prompt(instruction = 'Please translate the response above to Portuguese.')

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Portuguese. 
 Olá! Como posso ajudar?

In [67]:
prompt(instruction='Please translate the response above to Chinese.')

Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 Please translate the response above to Chinese. 
 你好！我能为你做些什么？

### 1

## Calling Custom User Defined Functions
- all the built in commands are functions from guidance.library.* but you can also pass in your own functions
    - Directly embbed, input & Output should be text

In [ ]:
# all the built in commands are functions from guidance.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''')
executed_program = program(aggregate=aggregate)

## Chat

In [68]:
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

### Role tags

In [ ]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='What is the meaning of life?')

In [69]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')
program()

Stop program system You are a helpful assistant. user Is the following joke funny? Why or why not?
 Sure, here's a joke for you: Why did the tomato turn red? Because it saw the salad dressing! assistant As an AI language model, I don't have the ability to find jokes funny or not. However, I can analyze the joke structure and language. This joke is a play on words and relies on a pun. The punchline is unexpected and relies on the double meaning of the word "dressing." Some people may find it funny, while others may not. It ultimately depends on personal taste and sense of humor.

### Agents

In [70]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [71]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {}]

In [72]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

Stop program system You are a helpful assistant user hi there assistant Hello! How can I assist you today? user What is the meaning of life? assistant The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that the meaning of life is to seek spiritual enlightenment or to make a positive impact on the world. Ultimately, the meaning of life is a deeply personal and subjective question that each individual must answer for themselves. user {{set 'conversation[-1].user_text' (await 'user_text')}} {{#assistant~}} 
 {{gen 'conversation[-1].ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~#geneach 'conversation' stop=False}} 
 {{#user~}} 
 {{set 'this.user_text' (await 'user_text')}} 
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'this.ai_text' temperature=0 max_tokens=300}} 
 {{~/assistant}} 
 {{~/geneach}}

In [73]:
program['conversation']

[{'user_text': 'hi there', 'ai_text': 'Hello! How can I assist you today?'},
 {'user_text': 'What is the meaning of life?',
  'ai_text': "The meaning of life is a philosophical question that has been debated by scholars, theologians, and thinkers for centuries. There is no one definitive answer to this question, as it can be interpreted in many different ways depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Still, others believe that the meaning of life is to seek spiritual enlightenment or to make a positive impact on the world. Ultimately, the meaning of life is a deeply personal and subjective question that each individual must answer for themselves."},
 {}]